In [202]:
from datasets import load_dataset
from PIL import Image

In [210]:
datasets = load_dataset("katanaml/cord", split='test')

Using the latest cached version of the module from /Users/aldofebri/.cache/huggingface/modules/datasets_modules/datasets/katanaml--cord/fa0960248a7d19cf19675785d5d3dd9eab83b4aea9274b97943d534be56d8a91 (last modified on Fri Jan 24 13:25:45 2025) since it couldn't be found locally at katanaml/cord, or remotely on the Hugging Face Hub.
Repo card metadata block was not found. Setting CardData to empty.


In [204]:
datasets[0]

{'id': '0',
 'words': ['TAX',
  '5.455',
  'TOTAL',
  '60.000',
  '(Qty',
  '2.00',
  'EDC',
  'CIMB',
  'NIAGA',
  'No:',
  'xx7730',
  '60.000',
  '901016',
  '-TICKET',
  'CP',
  '2',
  '60.000',
  '60.000',
  'Subtotal',
  '60.000',
  'TOTAL',
  'DISC',
  '$',
  '-60.000'],
 'bboxes': [[490, 743, 555, 774],
  [722, 746, 819, 777],
  [101, 827, 282, 858],
  [648, 820, 851, 856],
  [314, 820, 444, 856],
  [479, 822, 615, 851],
  [138, 904, 203, 938],
  [208, 904, 287, 935],
  [291, 901, 388, 935],
  [393, 901, 453, 932],
  [458, 898, 578, 929],
  [731, 901, 847, 932],
  [97, 604, 212, 635],
  [337, 595, 462, 626],
  [467, 595, 513, 626],
  [208, 641, 231, 669],
  [402, 632, 518, 663],
  [703, 632, 814, 663],
  [407, 783, 550, 814],
  [708, 783, 824, 817],
  [337, 709, 435, 740],
  [435, 709, 520, 736],
  [523, 709, 550, 734],
  [687, 709, 819, 737]],
 'ner_tags': [14,
  14,
  22,
  22,
  19,
  19,
  17,
  17,
  17,
  17,
  17,
  17,
  4,
  3,
  3,
  1,
  5,
  0,
  13,
  13,
  10,
  1

In [205]:
import torch
from transformers import LayoutLMv2Processor, LayoutLMv2ForTokenClassification

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
model = LayoutLMv2ForTokenClassification.from_pretrained("katanaml/layoutlmv2-finetuned-cord")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [206]:
import numpy as np
expected_list = []
predicted_list = []
for data in datasets:
    image_file = data['image_path'].split("image/")[-1]
    # image_file = 'receipt_00012.png'
    print(image_file)
    image = Image.open("image/" + image_file)
    words = data['words']
    bbox = data['bboxes']
    encoded_inputs = processor(image, words, boxes=bbox, return_offsets_mapping=True, return_tensors="pt")
    offset_mapping = encoded_inputs.pop('offset_mapping')
    for k,v in encoded_inputs.items():
        encoded_inputs[k] = v.to(device)
    # load the fine-tuned model from the hub
    id2label = model.config.id2label
    model.to(device)

    # forward pass
    outputs = model(**encoded_inputs)
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    is_subword = np.array(offset_mapping.squeeze().tolist())[:,0] != 0
    true_predictions = [pred for idx, pred in enumerate(predictions) if not is_subword[idx]][1:-1]
    expected = data['ner_tags']
    if len(expected) != len(true_predictions):
        continue
        print(f"Length mismatch: Expected {len(expected)} tokens, but got {len(true_predictions)} tokens, {image_file}")
    else:
        expected_list.extend(expected)
        predicted_list.extend(true_predictions)
    # Break after first iteration for debugging purposes
    # break;

receipt_00000.png
receipt_00001.png
receipt_00002.png
receipt_00003.png
receipt_00004.png
receipt_00005.png
receipt_00006.png
receipt_00007.png
receipt_00008.png
receipt_00009.png
receipt_00010.png
receipt_00011.png
receipt_00012.png


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


receipt_00013.png
receipt_00014.png
receipt_00015.png
receipt_00016.png
receipt_00017.png
receipt_00018.png
receipt_00019.png
receipt_00020.png
receipt_00021.png
receipt_00022.png
receipt_00023.png
receipt_00024.png
receipt_00025.png
receipt_00026.png
receipt_00027.png
receipt_00028.png
receipt_00029.png
receipt_00030.png
receipt_00031.png
receipt_00032.png
receipt_00033.png
receipt_00034.png
receipt_00035.png
receipt_00036.png
receipt_00037.png
receipt_00038.png
receipt_00039.png
receipt_00040.png
receipt_00041.png
receipt_00042.png
receipt_00043.png
receipt_00044.png
receipt_00045.png
receipt_00046.png
receipt_00047.png
receipt_00048.png
receipt_00049.png
receipt_00050.png
receipt_00051.png
receipt_00052.png
receipt_00053.png
receipt_00054.png
receipt_00055.png
receipt_00056.png
receipt_00057.png
receipt_00058.png
receipt_00059.png
receipt_00060.png
receipt_00061.png
receipt_00062.png
receipt_00063.png
receipt_00064.png
receipt_00065.png
receipt_00066.png
receipt_00067.png
receipt_00

In [207]:
print(len(expected_list), len(predicted_list))

2299 2299


In [208]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(expected_list, predicted_list)

TP, FP, FN, TN = [], [], [], []

for i in range(len(conf_matrix)):
    tp = conf_matrix[i, i]
    TP.append(tp)
    
    fp = conf_matrix[:, i].sum() - tp
    FP.append(fp)
    
    fn = conf_matrix[i, :].sum() - tp
    FN.append(fn)
    
    tn = conf_matrix.sum() - (conf_matrix[i, :].sum() + conf_matrix[:, i].sum() - tp)
    TN.append(tn)
print(tp, fp, fn, tn)

204 5 1 2089


In [209]:
def calculate_accuracy(tp, fp, fn, tn):
    return (tp + tn) / (tp + fp + fn + tn)

def calculate_precision(tp, fp):
    return tp / (tp + fp) if (tp + fp) != 0 else 0

def calculate_recall(tp, fn):
    return tp / (tp + fn) if (tp + fn) != 0 else 0

def calculate_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

acc = calculate_accuracy(tp, fp, fn, tn)
prec = calculate_precision(tp, fp)
recall = calculate_recall(tp, fn)
f1 = calculate_f1(prec, recall)
print(acc, prec, recall, f1)

0.9973901696389734 0.9760765550239234 0.9951219512195122 0.9855072463768115
